# 코드 실행 가이드
- 아래 모든 코드는 !(느낌표)를 제외하고 터미널에 입력하여도 그대로 실행할 수 있습니다.
```bash
python -m utils.split_data -h
```

## train 데이터셋 분할
- 원본 데이터셋을 학습에 활용할 수 있도록 train / validation으로 나눕니다.
- 필요에 따라 해당 스크립트를 사용하지 않고도 직접 train.csv, validation.csv를 만들어 사용할 수 있습니다.

In [ ]:
# 도움말 확인
!python utils/split_data.py --help

In [ ]:
# 데이터셋 분할 실행 (모든 데이터셋 사용, 70:30 split)
!python utils/split_data.py --datasets all --train-ratio 0.8

## 모델 훈련
- 분할한 데이터셋으로 모델을 훈련합니다.
- `model_definitions/`에 모델을 구현하고, 등록한 후 사용할 수 있습니다.
- 기본 baseline은 모든 학생의 제출을 목표로 합니다.
- 터미널에서 직접 실행 시 다음과 같이 입력하세요.
```bash
python -m train --model bilstm --config configs/bilstm.yaml --device cuda
```

In [ ]:
# 모델 학습 실행
model_name = "bilstm"
config_path = "configs/bilstm.yaml"
device = "cuda"  # 또는 "cpu"

# 학습 실행
!python train.py --model {model_name} --config {config_path} --device {device}

In [ ]:
# 생성된 모델 파일 확인
import os
from pathlib import Path

models_dir = Path("models")
if models_dir.exists():
    model_files = list(models_dir.glob("*.pt"))
    print(f"생성된 모델 파일:")
    for f in sorted(model_files):
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  - {f.name}: {size_mb:.2f} MB")
else:
    print("models 디렉토리가 없습니다.")


## 4. API 서버 실행

학습 완료 후 API 서버를 실행하여 모델을 테스트할 수 있습니다.

### 별도 터미널에서 실행:
```bash
# 방법 1: uvicorn 직접 실행
uvicorn app:app --host 0.0.0.0 --port 8000

# 방법 2: Python으로 실행
python app.py
```

### API 엔드포인트:
- `POST /reload_model`: 모델 리로드 (학습 후 반드시 실행)
- `POST /validate`: 검증 데이터셋으로 성능 평가
- `POST /infer`: 단일 텍스트 예측
- `POST /infer_csv`: CSV 파일 일괄 예측

### Swagger UI 접속:
서버 실행 후 브라우저에서 접속:
- Swagger UI: `http://localhost:8000/docs`
- ReDoc: `http://localhost:8000/redoc`

**⚠️ 주의:** API Key 필요 (`X-API-Key` 헤더에 `ULmLAYYhKeeP9J1c` 입력)


In [ ]:
## 3. 학습 결과 확인

학습이 완료되면 다음 파일들이 생성됩니다:
- `models/bilstm_best.pt`: 각 epoch별 최고 성능 모델
- `models/best.pt`: 최종 제출용 모델 (자동 복사됨)
